In [607]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [608]:
#define the url
url = 'https://www.rebellionfestivals.com/line-up'
#get the html
html = requests.get(url)
#parse the html
soup = BeautifulSoup(html.text, 'html.parser')

In [1157]:
#find lineup_item_artist 
lineup_item_artist = soup.find_all('div', class_='lineup__item')
lineup_item_artist[0].text
weekdays = ['Thursday', 'Friday', 'Saturday', 'Sunday']
stage = []
for i in range(len(lineup_item_artist)):
    stage.append(lineup_item_artist[i].text.split('\n'))
flat_list = [item for sublist in stage for item in sublist]
df = pd.DataFrame(flat_list, columns=['stage'])
df.reset_index(inplace=True, drop=True)
df = df.astype('str')
df['date'] = 0
df['artist'] = 0
df['start_time'] = 0
df['end_time'] = 0
df['show_room'] = 0
df.head()

#get the dates of the shows
dates = soup.find_all('h3', class_="lineup__item__date__heading")
dates[0].text
dates_list= []
for i in dates:
    dates_list.append(i.text)
dates_list = dates_list[0:4]

#get the artists of the shows
artists = soup.find_all('p', class_="lineup__item__artist")
artist_list = []
for i in artists:
    artist_list.append(i.text)

#get the stages of the shows
stages = soup.find_all('h3', class_="lineup__item__stage-heading")
stages[2].text
stages_list = []
for i in stages:
    stages_list.append(i.text)

times = soup.find_all('span', class_="lineup__item__artist__time")
times
times_list = []
for i in times:
    times_list.append(re.findall('[0-9]+:[0-9]+', i.text))






In [1158]:
df.head()

,stage,date,artist,start_time,end_time,show_room
0,R-FEST,0,0,0,0,0
1,Thursday4th August,0,0,0,0,0
2,21:30 - 22:45LEVELLERS,0,0,0,0,0
3,19:55 - 21:00HAWKWIND,0,0,0,0,0
4,18:25 - 19:25MISTY IN ROOTS,0,0,0,0,0


In [1159]:

#match all the data
for i in range(len(df)):
    if df.stage[i] in stages_list:
        df.show_room[i] = df.stage[i]
    if df.stage[i] in dates_list:
        df.date[i] = re.sub(r"([0-9]+(\.[0-9]+)?)th",r" \1 ", df.stage[i]).strip()
    if df.stage[i] in artist_list:
        df.artist[i] = df.stage[i].replace(str(re.findall('[0-9]+:[0-9]+ - [0-9]+:[0-9]+', df.stage[i])[0]),'')
        
    if re.findall('[0-9]+:[0-9]+', df.stage[i]) in times_list:
        df.start_time[i] = re.findall('[0-9]+:[0-9]+', df.stage[i])[0]
        df.end_time[i] = re.findall('[0-9]+:[0-9]+', df.stage[i])[1]
df.drop('stage', inplace=True, axis=1) 

df.head(40)

<ipython-input-1159-0751cf4be6f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.show_room[i] = df.stage[i]
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-1159-0751cf4be6f6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.date[i] = re.sub(r"([0-9]+(\.[0-9]+)?)th",r" \1 ", df.stage[i]).strip()


,date,artist,start_time,end_time,show_room
0,0,0,0,0,R-FEST
1,Thursday 4 August,0,0,0,0
2,0,LEVELLERS,21:30,22:45,0
3,0,HAWKWIND,19:55,21:00,0
4,0,MISTY IN ROOTS,18:25,19:25,0
5,0,DREADZONE,17:05,17:55,0
6,0,FEROCIOUS DOG,15:45,16:35,0
7,0,BEANS ON TOAST,14:30,15:20,0
8,0,MILLIE MANDERS & THE SHUT UP,13:30,14:10,0
9,Friday 5 August,0,0,0,0


In [1160]:
df.date.replace(0,method='ffill', inplace=True)
df.show_room.replace(0,method='ffill', inplace=True)
df = df.replace(0,np.nan)
df.dropna(inplace=True)
df.start_time = pd.to_datetime(df.start_time, format='%H:%M').dt.time
df.end_time = pd.to_datetime(df.end_time, format='%H:%M').dt.time
df.date.replace('  ', ' ', regex=True, inplace=True)
df.reset_index(inplace=True, drop=True)
#fix the dates
replace_dates= {'Thursday 4 August':'2022-08-04', 'Friday 5 August':'2022-08-05','Saturday 6 August':'2022-08-06','Sunday 7 August':'2022-08-07'}
df.date.replace(replace_dates, inplace=True)
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
df = df.rename(columns={'show_room':'stage'})
df.head(40)

,date,artist,start_time,end_time,stage
0,2022-08-04,LEVELLERS,21:30:00,22:45:00,R-FEST
1,2022-08-04,HAWKWIND,19:55:00,21:00:00,R-FEST
2,2022-08-04,MISTY IN ROOTS,18:25:00,19:25:00,R-FEST
3,2022-08-04,DREADZONE,17:05:00,17:55:00,R-FEST
4,2022-08-04,FEROCIOUS DOG,15:45:00,16:35:00,R-FEST
5,2022-08-04,BEANS ON TOAST,14:30:00,15:20:00,R-FEST
6,2022-08-04,MILLIE MANDERS & THE SHUT UP,13:30:00,14:10:00,R-FEST
7,2022-08-05,THE STRANGLERS,21:25:00,22:45:00,R-FEST
8,2022-08-05,THE UNDERTONES,19:55:00,20:55:00,R-FEST
9,2022-08-05,SKIDS,18:25:00,19:25:00,R-FEST


In [1161]:
new_df = pd.DataFrame(columns=[stages_list], index=[df.date,df.start_time])
new_df.head()


R-FEST EMPRESS BALLROOM CLUB CASBAH PAVILION  \
date       start_time                                                
2022-08-04 21:30:00      NaN              NaN         NaN      NaN   
           19:55:00      NaN              NaN         NaN      NaN   
           18:25:00      NaN              NaN         NaN      NaN   
           17:05:00      NaN              NaN         NaN      NaN   
           15:45:00      NaN              NaN         NaN      NaN   

                      AFTER DARK & RIS LITERARY ALMOST ACOUSTIC  
date       start_time                                            
2022-08-04 21:30:00                NaN      NaN             NaN  
           19:55:00                NaN      NaN             NaN  
           18:25:00                NaN      NaN             NaN  
           17:05:00                NaN      NaN             NaN  
           15:45:00                NaN      NaN             NaN

In [1162]:
for i in range(len(new_df)):
    #put artist name where condition is met
    if df.iloc[i].date in new_df.iloc[i].name and df.iloc[i].stage in new_df.iloc[i].index:
        new_df.iloc[i][df.iloc[i].stage] = df.iloc[i].artist + ': '+str(df.iloc[i].start_time)+'-'+str(df.iloc[i].end_time)

In [1163]:
new_df = new_df.groupby(['date','start_time']).apply(lambda x: x.ffill().bfill())

In [1164]:
new_df.sort_index(inplace=True)

In [1165]:
#remove duplicate rows from new_df
new_df.drop_duplicates(inplace=True)

In [1166]:
new_df.head(40)

R-FEST  \
date       start_time                                                    
2022-08-04 00:00:00                                                NaN   
           00:30:00                                                NaN   
           01:05:00                                                NaN   
           01:15:00                                                NaN   
           11:45:00                                                NaN   
           11:55:00                                                NaN   
           12:20:00                                                NaN   
           12:30:00                                                NaN   
           12:40:00                                                NaN   
           12:45:00                                                NaN   
           13:00:00                                                NaN   
           13:05:00                                                NaN   
           13:10:00                                                NaN   
           13:15:00                                                NaN   
           13:25:00                                                NaN   
           13:30:00    MILLIE MANDERS & THE SHUT UP: 13:30:00-14:10:00   
           13:40:00                                                NaN   
           13:50:00                                                NaN   
           13:55:00                                                NaN   
           14:00:00                                                NaN   
           14:10:00                                                NaN   
           14:15:00                                                NaN   
           14:30:00                  BEANS ON TOAST: 14:30:00-15:20:00   
           14:35:00                                                NaN   
           14:45:00                                                NaN   
           14:50:00                                                NaN   
           15:10:00                                                NaN   
           15:30:00                                                NaN   
           15:35:00                                                NaN   
           15:45:00                   FEROCIOUS DOG: 15:45:00-16:35:00   
           15:50:00                                                NaN   
           16:05:00                                                NaN   
           16:10:00                                                NaN   
           16:15:00                                                NaN   
           16:25:00                                                NaN   
           16:35:00                                                NaN   
           16:50:00                                                NaN   
           17:00:00                                                NaN   
           17:05:00                       DREADZONE: 17:05:00-17:55:00   
           17:15:00                                                NaN   

                                               EMPRESS BALLROOM  \
date       start_time                                             
2022-08-04 00:00:00                                         NaN   
           00:30:00                                         NaN   
           01:05:00                                         NaN   
           01:15:00                                         NaN   
           11:45:00                                         NaN   
           11:55:00                                         NaN   
           12:20:00                                         NaN   
           12:30:00                                         NaN   
           12:40:00                                         NaN   
           12:45:00              JANUS STARK: 12:45:00-13:20:00   
           13:00:00                                         NaN   
           13:05:00                                         NaN   
           13:10:

In [1167]:
#save new df to excel
new_df.to_excel('rebellion_festival.xlsx')

In [1084]:
start_times = df.start_time.unique()
start_times = pd.DataFrame(start_times, columns=['start_time'])

In [1085]:
new_df = pd.merge(start_times, r_fest_df, on='start_time', how='left').sort_values('start_time')
new_df = pd.concat([new_df, empress_ballromm_df], keys=['date','start_time'], join='outer')

new_df.head(60)

start_time       date                            artist  end_time  \
date 85    00:00:00        NaT                               NaN       NaN   
     94    00:05:00        NaT                               NaN       NaN   
     37    00:15:00        NaT                               NaN       NaN   
     105   00:30:00        NaT                               NaN       NaN   
     84    01:05:00        NaT                               NaN       NaN   
     93    01:15:00        NaT                               NaN       NaN   
     111   11:45:00        NaT                               NaN       NaN   
     64    11:55:00        NaT                               NaN       NaN   
     92    12:20:00        NaT                               NaN       NaN   
     72    12:30:00        NaT                               NaN       NaN   
     52    12:35:00        NaT                               NaN       NaN   
     44    12:40:00        NaT                               NaN       NaN   
     36    12:45:00        NaT                               NaN       NaN   
     83    13:00:00        NaT                               NaN       NaN   
     91    13:05:00        NaT                               NaN       NaN   
     129   13:10:00        NaT                               NaN       NaN   
     71    13:15:00        NaT                               NaN       NaN   
     51    13:25:00        NaT                               NaN       NaN   
     16    13:30:00 2022-08-04      MILLIE MANDERS & THE SHUT UP  14:10:00   
     17    13:30:00 2022-08-05                        THE VAPORS  14:10:00   
     18    13:30:00 2022-08-06  PETE BENTHAM & THE DINNER LADIES  14:10:00   
     19    13:30:00 2022-08-07                          I, DORIS  14:10:00   
     57    13:35:00        NaT                               NaN       NaN   
     35    13:40:00        NaT                               NaN       NaN   
     82    13:45:00        NaT                               NaN       NaN   
     128   13:50:00        NaT                               NaN       NaN   
     90    13:55:00        NaT                               NaN       NaN   
     70    14:00:00        NaT                               NaN       NaN   
     79    14:05:00        NaT                               NaN       NaN   
     115   14:10:00        NaT                               NaN       NaN   
     63    14:15:00        NaT                               NaN       NaN   
     43    14:20:00        NaT                               NaN       NaN   
     50    14:25:00        NaT                               NaN       NaN   
     15    14:30:00 2022-08-07                    ALTERED IMAGES  15:20:00   
     14    14:30:00 2022-08-06                    THE PRIMITIVES  15:25:00   
     13    14:30:00 2022-08-05                       JILTED JOHN  15:15:00   
     12    14:30:00 2022-08-04                    BEANS ON TOAST  15:20:00   
     34    14:35:00        NaT                               NaN       NaN   
     89    14:45:00        NaT                               NaN       NaN   
     114   14:50:00        NaT                               NaN       NaN   
     69    14:55:00        NaT                               NaN       NaN   
     103   15:05:00        NaT                               NaN       NaN   
     62    15:10:00        NaT                               NaN       NaN   
     42    15:20:00        NaT                               NaN       NaN   
     119   15:25:00        NaT                               NaN       NaN   
     33    15:30:00        NaT                               NaN       NaN   
     22    15:35:00 2022-08-05                            TOYAH   16:30:00   
     124   15:40:00        NaT                               NaN       NaN   
     10    15:45:00 2022-08-06               THE WEDDING PRESENT  16:40:00   
     9     15:45:00 2022-08-04                     FEROCIOUS DOG  16:35:00   
     11    15:45:00 2022-08-07 

In [1086]:
#save the data to excel
df.to_excel('rebellion_festival.xlsx')